In [2]:
import numpy as np
import pandas as pd
import h5py
import os
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout, Activation, LeakyReLU
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# === [📂 Load HDF5 Data] ===
file_path = 'data/bitcoin2018_close.h5'
with h5py.File(file_path, 'r') as hf:
    datas = hf['inputs'][:]
    labels = hf['outputs'][:]
    input_times = hf['input_times'][:]
    output_times = hf['output_times'][:]
    original_inputs = hf['original_inputs'][:]
    original_outputs = hf['original_outputs'][:]
    original_datas = hf['original_datas'][:]

# === [🧪 Prepare Splits] ===
training_size = int(0.8 * datas.shape[0])
training_datas = datas[:training_size]
training_labels = labels[:training_size, :, 0]
validation_datas = datas[training_size:]
validation_labels = labels[training_size:, :, 0]
validation_original_outputs = original_outputs[training_size:]
validation_original_inputs = original_inputs[training_size:]
validation_input_times = input_times[training_size:]
validation_output_times = output_times[training_size:]

# === [⚙️ Model Parameters] ===
step_size = datas.shape[1]
nb_features = datas.shape[2]
output_size = labels.shape[1]
units = 50
epochs = 30
batch_size = 8

# === [🧠 GRU Model Function] ===
def fit_gru(reg):
    model = Sequential()
    model.add(GRU(units=units, kernel_regularizer=reg, input_shape=(step_size, nb_features), return_sequences=False))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(output_size))
    model.add(LeakyReLU())
    model.compile(loss='mse', optimizer='adam')
    model.fit(training_datas, training_labels, batch_size=batch_size, epochs=epochs, verbose=0)
    return model

# === [📊 Evaluation Function] ===
def experiment(reg, nb_repeat=5):
    scaler = MinMaxScaler()
    scaler.fit(original_datas[:, 0].reshape(-1, 1))
    scores = []
    for i in range(nb_repeat):
        model = fit_gru(reg)
        predicted = model.predict(validation_datas)
        predicted_inverted = scaler.inverse_transform(predicted).reshape(-1)
        actual = validation_original_outputs[:, :, 0].reshape(-1)
        scores.append(mean_squared_error(actual, predicted_inverted))
    return scores

# === [🔁 Run for Regularization Values] ===
regs = [
    regularizers.l1(0.0),
    regularizers.l1(0.01),
    regularizers.l1(0.001),
    regularizers.l2(0.01),
    regularizers.l2(0.001)
]

results = pd.DataFrame()
for reg in regs:
    if hasattr(reg, 'l1'):
        name = f"l1 {reg.l1:.4f}, l2 0.0000"
    else:
        name = f"l1 0.0000, l2 {reg.l2:.4f}"

    print(f"Training model with: {name}")
    scores = experiment(reg, nb_repeat=5)
    results[name] = scores

# === [💾 Save & Show Results] ===
if not os.path.exists('result'):
    os.makedirs('result')

results.describe().to_csv('result/gru_regularization_results.csv')
print(results.describe())

Training model with: l1 0.0000, l2 0.0000


c:\Users\Prakriti\Desktop\crypto\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


c:\Users\Prakriti\Desktop\crypto\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step


c:\Users\Prakriti\Desktop\crypto\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


c:\Users\Prakriti\Desktop\crypto\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
       l1 0.0000, l2 0.0000  l1 0.0100, l2 0.0000  l1 0.0010, l2 0.0000  \
count              5.000000          5.000000e+00              5.000000   
mean          140566.126571          1.094417e+06         292537.131559   
std           148071.248429          7.388597e+05          49778.300934   
min            12135.529807          2.719796e+05         220614.969783   
25%            49763.833570          3.193008e+05         263120.082457   
50%            84481.236701          1.454051e+06         311198.829894   
75%           173545.865930          1.639607e+06         326550.851376   
max           382904.166848          1.787149e+06         341200.924285   

       l1 0.0000, l2 0.0100  l1 0.0000, l2 0.0010  
count              5.000000              5.000000  
mean          229823.995797          72640.695617  
std           157232.536288          63422.828879  
min            65283.047033          19418.610499  
25%            82722.51449